[Reference](https://medium.com/swlh/5-tips-for-working-with-time-series-in-python-d889109e676d)

In [1]:
def fft_denoiser(x, n_components, to_real=True):
    """Fast fourier transform denoiser.
    
    Denoises data using the fast fourier transform.
    
    Parameters
    ----------
    x : numpy.array
        The data to denoise.
    n_components : int
        The value above which the coefficients will be kept.
    to_real : bool, optional, default: True
        Whether to remove the complex part (True) or not (False)
        
    Returns
    -------
    clean_data : numpy.array
        The denoised data.
        
    References
    ----------
    .. [1] Steve Brunton - Denoising Data with FFT[Python]
       https://www.youtube.com/watch?v=s2K1JfNR7Sc&ab_channel=SteveBrunton
    
    """
    n = len(x)
    
    # compute the fft
    fft = np.fft.fft(x, n)
    
    # compute power spectrum density
    # squared magnitud of each fft coefficient
    PSD = fft * np.conj(fft) / n
    
    # keep high frequencies
    _mask = PSD > n_components
    fft = _mask * fft
    
    # inverse fourier transform
    clean_data = np.fft.ifft(fft)
    
    if to_real:
        clean_data = clean_data.real
    
    return clean_data

In [2]:
def basic_filter(data, mode='rolling', window=262, threshold=3):
    """Basic Filter.
    
    Mark as outliers the points that are out of the interval:
    (mean - threshold * std, mean + threshold * std ).
    
    Parameters
    ----------
    data : pandas.Series
        The time series to filter.
    mode : str, optional, default: 'rolling'
        Whether to filter in rolling or expanding basis.
    window : int, optional, default: 262
        The number of periods to compute the mean and standard
        deviation.
    threshold : int, optional, default: 3
        The number of standard deviations above the mean.
        
    Returns
    -------
    series : pandas.DataFrame
        Original series and marked outliers.
    """
    msg = f"Type must be of pandas.Series but {type(data)} was passed."
    assert isinstance(data, pd.Series), msg
    
    series = data.copy()
    
    # rolling/expanding objects
    pd_object = getattr(series, mode)(window=window)
    mean = pd_object.mean()
    std = pd_object.std()
    
    upper_bound = mean + threshold * std
    lower_bound = mean - threshold * std
    
    outliers = ~series.between(lower_bound, upper_bound)
    # fill false positives with 0
    outliers.iloc[:window] = np.zeros(shape=window)
    
    series = series.to_frame()
    series['outliers'] = np.array(outliers.astype('int').values)
    series.columns = ['Close', 'Outliers']
    
    return series

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin


class RollingStandardScaler(BaseEstimator, TransformerMixin):
    """Rolling standard Scaler
    
    Standardized the given data series using the mean and std 
    commputed in rolling or expanding mode.
    
    Parameters
    ----------
    window : int
        Number of periods to compute the mean and std.
    mode : str, optional, default: 'rolling'
        Mode 
        
    Attributes
    ----------
    pd_object : pandas.Rolling
        Pandas window object.
    w_mean : pandas.Series
        Series of mean values.
    w_std : pandas.Series
        Series of std. values.
    """
    def __init__(self, window, mode='rolling'):
        self.window = window
        self.mode = mode
        
        # to fill in code
        self.pd_object = None
        self.w_mean = None
        self.w_std = None
        self.__fitted__ = False
        
    def __repr__(self):
        return f"RollingStandardScaler(window={self.window}, mode={self.mode})"
        
    def fit(self, X, y=None):
        """Fits.
        
        Computes the mean and std to be used for later scaling.
        
        Parameters
        ----------
        X : array-like of shape (n_shape, n_features)
            The data used to compute the per-feature mean and std. Used for
            later scaling along the feature axis.
        y
            Ignored.
        """
        self.pd_object = getattr(X, self.mode)(self.window)
        self.w_mean = self.pd_object.mean()
        self.w_std = self.pd_object.std()
        self.__fitted__ = True
        
        return self
    
    def transform(self, X):
        """Transforms.
        
        Scale features of X according to the window mean and standard 
        deviation.
        
        Paramaters
        ----------
        X : array-like of shape (n_shape, n_features)
            Input data that will be transformed.
        
        Returns
        -------
        standardized : array-like of shape (n_shape, n_features)
            Transformed data.
        """
        self._check_fitted()
        
        standardized = X.copy()
        return (standardized - self.w_mean) / self.w_std
    
    def inverse_transform(self, X):
        """Inverse transform
        
        Undo the transform operation
        
        Paramaters
        ----------
        X : array-like of shape (n_shape, n_features)
            Input data that will be transformed.
        
        Returns
        -------
        standardized : array-like of shape (n_shape, n_features)
            Transformed (original) data.
        """
        self._check_fitted()
        
        unstandardized = X.copy()
        return  (unstandardized * self.w_std) + self.w_mean
        
    def _check_fitted(self):
        """ Checks if the algorithm is fitted. """
        if not self.__fitted__:
            raise ValueError("Please, fit the algorithm first.")

In [4]:
""" Utils functions. """

import numpy as np
import pandas as pd


def compute_returns(data, periods=1, log=False, relative=True):
    """Computes returns.
    Calculates the returns of a given dataframe for the given period. The 
    returns can be computed as log returns or as arithmetic returns
    
    Parameters
    ----------
    data : pandas.DataFrame or pandas.Series
        The data to calculate returns of.
    periods : int
        The period difference to compute returns.
    log : bool, optional, default: False
        Whether to compute log returns (True) or not (False).
    relative : bool, optional, default: True
        Whether to compute relative returns (True) or not 
        (False).
    
    Returns
    -------
    ret : pandas.DataFrame or pandas.Series
        The computed returns.
    
    """
    if log:
        if not relative:
            raise ValueError("Log returns are relative by definition.")
        else:
            ret = _log_returns(data, periods)
    else:
        ret = _arithmetic_returns(data, periods, relative)

    return ret


def _arithmetic_returns(data, periods, relative):
    """Arithmetic returns."""
    # to avoid computing it twice
    shifted = data.shift(periods)
    ret = (data - shifted) 
    
    if relative:
        return ret / shifted
    else:
        return ret


def _log_returns(data, periods):
    """Log returns."""
    return np.log(data / data.shift(periods))